## Установка Clickhouse-driver

In [ ]:
pip install clickhouse-driver

## Импорт бибилиотек

In [ ]:
from clickhouse_driver import Client
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

Подключение к базде данных

In [ ]:
client = Client(host = "dev-01.aksimemor.me",
                port = 9000,
                database = "default",
                user = "thanos",
                password = "show_me_your_data")

Запрос в БД таблицу Marvel для просмотра первых 3 записей и преобразхование в DF будем делать за счет следующего кода

In [ ]:
top3, columns = client.execute('SELECT * FROM marvel LIMIT 3', 
                                 {'species': "Iris-setosa"},
                                 with_column_types=True)
top3 = pd.DataFrame(top3, columns=[tuple[0] for tuple in columns])
top3

In [ ]:
top3.info()

## Описание данных
user_id - id игрока  
installed_at - Дата и время установки  
created_at - Дата и время события  
day - День жизни игрока с момента установки  
country - Страна пользователя в момент совершения   события
app_version_short - Версия мобильного приложения  
last_time_spent - Продолжительность последней сессии в   секундах  
lifetime_session_count - Общее кол-во сессий игрока к данному событию  
activity_kind - Тип события  
event_name - Название события  
reporting_revenue - Выручка  

## ARPU
ARPU (по дням, неделям и месяцам) - средний доход с одного активного пользователя

### Запрос в SQL: 
    SELECT     
        toYYYYMM(installed_at) as dateIn, 
        toYYYYMM(created_at) as dateCr,   
        dateDiff(month, installed_at, created_at) as Month,  
        COUNT(DISTINCT(user_id)) as ordersCount,   
        SUM(reporting_revenue) as sumRev,   
        (SUM(reporting_revenue)/COUNT(DISTINCT(user_id))) as ARPUMonth    
    FROM marvel   
    GROUP BY dateIn, dateCr, Month
    
Для недельных и ежедневных данных заменим dateDiff на нужные когорты

## ARPU Month

In [ ]:
ARPUmonth, columns = client.execute('SELECT toYYYYMM(installed_at) as dateIn, toYYYYMM(created_at) as dateCr, dateDiff(month, installed_at, created_at) as Month, COUNT(DISTINCT(user_id)) as ordersCount, SUM(reporting_revenue) as sumRev, (SUM(reporting_revenue)/COUNT(DISTINCT(user_id))) as ARPUMonth  FROM marvel GROUP BY dateIn, dateCr, Month',
                                 {'species': "Iris-setosa"},
                                 with_column_types=True)
ARPUmonth = pd.DataFrame(ARPUmonth, columns=[tuple[0] for tuple in columns])
CohortARPUmonth = ARPUmonth.pivot_table(index='dateIn', columns='Month', values='ARPUMonth')
CohortARPUmonth.fillna(0, inplace=True)

#График когорт
CohortARPUmonth.T.plot(figsize=(16, 8))
plt.title('ARPUMonth')
plt.ylabel('ARPU')

#Матрица когорты
sns.set(style='white')
plt.figure(figsize=(16, 8))
plt.title('ARPUMonth')
sns.heatmap(CohortARPUmonth, annot=True, fmt='.2f')

## ARPU Week

In [ ]:
ARPUWeek, columns = client.execute('SELECT toYearWeek(installed_at) as dateIn, toYearWeek(created_at) as dateCr, dateDiff(week, installed_at, created_at) as Week, COUNT(DISTINCT(user_id)) as ordersCount, SUM(reporting_revenue) as sumRev, (SUM(reporting_revenue)/COUNT(DISTINCT(user_id))) as ARPUWeek FROM marvel GROUP BY dateIn, dateCr, Week',
                                 {'species': "Iris-setosa"},
                                 with_column_types=True)
ARPUweek = pd.DataFrame(ARPUWeek, columns=[tuple[0] for tuple in columns])
CohortARPUWeek = ARPUweek.pivot_table(index='dateIn', columns='Week', values='ARPUWeek')
CohortARPUWeek.fillna(0, inplace=True)

#График когорт
CohortARPUWeek.T.plot(figsize=(16, 8))
plt.title('ARPUMWeek')
plt.ylabel('ARPU')

#Матрица когорты (Большйо размер неудобно отображать!)
#sns.set(style='white')
#plt.figure(figsize=(16, 8))
#plt.title('ARPUWeek')
#sns.heatmap(CohortARPUWeek, annot=True, fmt='.2f')

## ARPU Day

In [ ]:
ARPUDay, columns = client.execute('SELECT toYearWeek(installed_at) as dateIn, toYYYYMMDD(created_at) as dateCr, dateDiff(day, installed_at, created_at) as Day, COUNT(DISTINCT(user_id)) as ordersCount, SUM(reporting_revenue) as sumRev, (SUM(reporting_revenue)/COUNT(DISTINCT(user_id))) as ARPUDay  FROM marvel GROUP BY dateIn, dateCr, Day',
                                 {'species': "Iris-setosa"},
                                 with_column_types=True)
ARPUDay = pd.DataFrame(ARPUDay, columns=[tuple[0] for tuple in columns])
CohortARPUDay = ARPUDay.pivot_table(index='dateIn', columns='Day', values='ARPUDay')
CohortARPUDay.fillna(0, inplace=True)

#График когорт (Плохо видно, так как большая выборка данных)
CohortARPUDay.T.plot(figsize=(16, 8))
plt.title('ARPUDay')
plt.ylabel('ARPU')

#Матрица когорты (Большйо размер неудобно отображать!)
#sns.set(style='white')
#plt.figure(figsize=(16, 8))
#plt.title('ARPUDay')
#sns.heatmap(CohortARPUDay, annot=True, fmt='.2f')

## Небольшие итоги по ARPU:
Для некоторых когорт имеются выбросы, большей частью это первые когорты пользователи которые зарегестрирвоались. Причины скачков могут быть различными:  
случайность,  
праздники,  
киты (игроки которые тратят больше всего денег).

## Retention
Retention - это коэффициент удержания пользователей, показывающий «возвращаемость» клиентов в приложение за определенный период после установки.

### Запрос в SQL: 
    
    SELECT 
        T1.cohort_month, 
        T1.month_number, 
        T1.num_users, 
        T2.num_users, 
        T1.num_users/T2.num_users as retention 
    FROM 
         (SELECT 
         toYYYYMM(installed_at) as cohort_month,
         dateDiff(month, installed_at, created_at) as month_number,
         count(cohort_month) as num_users 
         FROM marvel 
         group by cohort_month, month_number) AS T1,
         (SELECT toYYYYMM(installed_at) as cohort_month, 
         count(cohort_month) as num_users 
         FROM marvel 
         GROUP BY cohort_month) AS T2 
    WHERE T1.cohort_month=T2.cohort_month ORDER BY T1.cohort_month'
    
Для недельных и ежедневных данных заменим dateDiff на нужные когорты

## Retention Month

In [ ]:
RetentionMonth, columns = client.execute('SELECT T1.cohort_month, T1.month_number, T1.num_users, T2.num_users, T1.num_users/T2.num_users as retention FROM (SELECT toYYYYMM(installed_at) as cohort_month, dateDiff(month, installed_at, created_at) as month_number, count(cohort_month) as num_users FROM marvel group by cohort_month, month_number) as T1, (SELECT toYYYYMM(installed_at) as cohort_month, count(cohort_month) as num_users FROM marvel group by cohort_month) as T2 WHERE T1.cohort_month=T2.cohort_month ORDER BY T1.cohort_month',
                                 with_column_types=True)
RetentionMonth = pd.DataFrame(RetentionMonth, columns=[tuple[0] for tuple in columns])
CohortRetentionMonth = RetentionMonth.pivot_table(index='cohort_month', columns='month_number', values='retention')
CohortRetentionMonth.fillna(0, inplace=True)


#График когорт (Плохо видно, так как большая выборка данных)
CohortRetentionMonth.T.plot(figsize=(16, 8))
plt.title('RetentionMonth')
plt.ylabel('Retention')

#Матрица когорты (Большйо размер неудобно отображать!)
sns.set(style='white')
plt.figure(figsize=(16, 8))
plt.title('RetentionMonth')
sns.heatmap(CohortRetentionMonth, annot=True, fmt='.2%')

## Retention Week

In [ ]:
RetentionWeek, columns = client.execute('SELECT T1.cohort_week, T1.week_number, T1.num_users, T2.num_users, T1.num_users/T2.num_users as retention FROM (SELECT toYearWeek(installed_at) as cohort_week, dateDiff(week, installed_at, created_at) as week_number, count(cohort_week) as num_users FROM marvel group by cohort_week, week_number) as T1, (SELECT toYearWeek(installed_at) as cohort_week, count(cohort_week) as num_users FROM marvel group by cohort_week) as T2 WHERE T1.cohort_week=T2.cohort_week ORDER BY T1.cohort_week',
                                 with_column_types=True)
RetentionWeek = pd.DataFrame(RetentionWeek, columns=[tuple[0] for tuple in columns])
CohortRetentionWeek = RetentionWeek.pivot_table(index='cohort_week', columns='week_number', values='retention')
CohortRetentionWeek.fillna(0, inplace=True)


#График когорт (Плохо видно, так как большая выборка данных)
CohortRetentionWeek.T.plot(figsize=(16, 8))
plt.title('RetentionWeek')
plt.ylabel('Retention')

#Матрица когорты (Большой размер неудобно отображать!)
#sns.set(style='white')
#plt.figure(figsize=(16, 8))
#plt.title('RetentionMonth')
#sns.heatmap(CohortRetentionWeek, annot=True, fmt='.2%')

## Retention Day

In [ ]:
RetentionDay, columns = client.execute('SELECT T1.cohort_day, T1.day_number, T1.num_users, T2.num_users, T1.num_users/T2.num_users as retention FROM (SELECT toYYYYMMDD(installed_at) as cohort_day, dateDiff(day, installed_at, created_at) as day_number, count(cohort_day) as num_users FROM marvel group by cohort_day, day_number) as T1, (SELECT toYYYYMMDD(installed_at) as cohort_day, count(cohort_day) as num_users FROM marvel group by cohort_day) as T2 WHERE T1.cohort_day=T2.cohort_day ORDER BY T1.cohort_day',
                                 with_column_types=True)
RetentionDay = pd.DataFrame(RetentionDay, columns=[tuple[0] for tuple in columns])
CohortRetentionDay = RetentionDay.pivot_table(index='cohort_day', columns='day_number', values='retention')
CohortRetentionDay.fillna(0, inplace=True)


#График когорт (Плохо видно, так как большая выборка данных)
CohortRetentionDay.T.plot(figsize=(16, 8))
plt.title('RetentionDay')
plt.ylabel('Retention')

#Матрица когорты (Большой размер неудобно отображать!)
#sns.set(style='white')
#plt.figure(figsize=(16, 8))
#plt.title('RetentionMonth')
#sns.heatmap(CohortRetentionDay, annot=True, fmt='.2%')

## Небольшие итоги по Retention:
Для некоторых когорт имеются выбросы. Причины скачков могут быть различными, но я думаю причина в ивентах.